In [1]:
install.packages('RSQLite', repos='https://cran.r-project.org/')
install.packages('sqldf', repos='https://cran.r-project.org/')

Installing package into 'C:/Users/User/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'RSQLite' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\User\AppData\Local\Temp\RtmpQjnTXW\downloaded_packages


Installing package into 'C:/Users/User/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'sqldf' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\User\AppData\Local\Temp\RtmpQjnTXW\downloaded_packages


1. Installing required packages

In [2]:
library(RSQLite)
library(sqldf)

Warning message:
"package 'RSQLite' was built under R version 3.3.3"Warning message:
"package 'sqldf' was built under R version 3.3.3"Loading required package: gsubfn
Warning message:
"package 'gsubfn' was built under R version 3.3.3"Loading required package: proto
Could not load tcltk.  Will use slower R code instead.


1. Adding the packages to the library

In [3]:
con = dbConnect(SQLite(), dbname="database.sqlite")
alltables = dbListTables(con)
show(alltables)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"  


1. Connect to the provided database using RSQLite dbconnect

In [4]:
playersData <- dbGetQuery( con,"SELECT DISTINCT REPLACE(P.player_name,',',' ') AS player_name, P.player_api_id, MAX(PA.overall_rating) AS overall_rating,
                MAX(PA.gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes) AS gk_attr_total, 
                Games.GamesPlayed AS games_played, Games.LeaguesPlayed AS leagues_played
                FROM Player P
                INNER JOIN Player_Attributes AS PA
                ON P.player_api_id=PA.player_api_id 
                INNER JOIN (
                SELECT P.player_name, P.player_api_id, COUNT(*) AS GamesPlayed, COUNT(DISTINCT M.league_id) AS LeaguesPlayed
					FROM Player P
					INNER JOIN Match AS M
					ON  M.away_player_1=P.player_api_id OR M.away_player_2=P.player_api_id OR M.away_player_3=P.player_api_id OR 
                        M.away_player_4=P.player_api_id OR M.away_player_5=P.player_api_id OR M.away_player_6=P.player_api_id OR 
                        M.away_player_7=P.player_api_id OR M.away_player_8=P.player_api_id OR M.away_player_9=P.player_api_id OR
                        M.away_player_10=P.player_api_id OR M.away_player_11=P.player_api_id OR M.home_player_1=P.player_api_id OR
                        M.home_player_2=P.player_api_id OR M.home_player_3=P.player_api_id OR M.home_player_4=P.player_api_id OR
                        M.home_player_5=P.player_api_id OR M.home_player_6=P.player_api_id OR M.home_player_7=P.player_api_id OR
                        M.home_player_8=P.player_api_id OR M.home_player_9=P.player_api_id OR M.home_player_10=P.player_api_id OR
                        M.home_player_11=P.player_api_id
					GROUP BY P.player_api_id
                    ORDER BY GamesPlayed DESC) AS Games
                ON Games.player_api_id = P.player_api_id
                GROUP BY P.player_api_id")

1. A cummulative query has been created to fetch all the relevant data related to Players.
2. The query fetchs player attributes like overall rating, total games played etc.

In [5]:
teamData <- dbGetQuery( con,"SELECT DISTINCT REPLACE(T.team_long_name,',',' ') AS team_name, T.team_api_id, GOALTABLE.TotalGoals AS total_goals_scored,
                MAX(IFNULL(TA.buildUpPlaySpeed,0) + IFNULL(TA.buildUpPlayDribbling,0) + IFNULL(TA.buildUpPlayPassing,0)) AS buildup_attr_total,
                MAX(IFNULL(TA.chanceCreationPassing,0) + IFNULL(TA.chanceCreationCrossing,0) + IFNULL(TA.chanceCreationShooting,0)) AS chance_creation_total,
                MAX(IFNULL(TA.defencePressure,0) + IFNULL(TA.defenceAggression,0) + IFNULL(TA.defenceTeamWidth,0)) AS defense_attr_total
                FROM Team T
                LEFT JOIN Team_Attributes AS TA
                ON T.team_api_id=TA.team_api_id
                INNER JOIN (SELECT DISTINCT T.team_long_name, T.team_api_id, 
                SUM(CASE 
                    WHEN T.team_api_id=M.away_team_api_id THEN M.away_team_goal ELSE M.home_team_goal END
                ) AS TotalGoals
                FROM Team T
                INNER JOIN Match AS M
                ON T.team_api_id=M.away_team_api_id OR T.team_api_id=M.home_team_api_id
                GROUP BY T.team_api_id) AS GOALTABLE
                ON GOALTABLE.team_api_id = T.team_api_id
                GROUP BY T.team_api_id")

1. A cummulative query has been created to fetch all the relevant data related to Teams.
2. The query fetchs attributes like total build up attributes, total goals scored etc.

In [6]:
write.csv(playersData, file = "Players.csv", row.names=FALSE)
write.csv(teamData, file = "Teams.csv", row.names=FALSE)

1. The queried results are written into a csv file for answering questions.